<a href="https://colab.research.google.com/github/Erynsia/Mirror-Torrent-To-Onedrives/blob/main/Mirror_Torrent_To_Onedrives.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!wget https://downloads.rclone.org/v1.64.2/rclone-v1.64.2-linux-amd64.deb
!apt install ./rclone-v1.64.2-linux-amd64.deb
!apt install fuse3

In [ ]:
!rclone config

In [ ]:
!mkdir onedrive
!mkdir downloads

In [ ]:
!rclone --vfs-cache-mode writes mount --allow-non-empty onedrive: ./onedrive &

In [ ]:
!ls ./onedrive

In [ ]:
link = input ("PASTE Magnet link / Torrent link")

import libtorrent as lt
import time
import datetime

ses = lt.session()
ses.listen_on(6881, 6891)
params = {
    'save_path': './downloads/',
    'storage_mode': lt.storage_mode_t(2)}

print(link)

handle = lt.add_magnet_uri(ses, link, params)
ses.start_dht()

begin = time.time()
print(datetime.datetime.now())

print ('Downloading Metadata...')
while (not handle.has_metadata()):
    time.sleep(1)
print ('Got Metadata, Starting Torrent Download...')

print("Starting", handle.name())

while (handle.status().state != lt.torrent_status.seeding):
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata', \
            'downloading', 'finished', 'seeding', 'allocating']
    print ('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' % \
            (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, \
            s.num_peers, state_str[s.state]))
    time.sleep(5)

end = time.time()
print(handle.name(), "COMPLETE")

print("Elapsed Time: ",int((end-begin)//60),"min :", int((end-begin)%60), "sec")

print(datetime.datetime.now())

In [ ]:
!rsync -ah --progress downloads/ onedrive/